In [31]:
#SHOULD MAKE BLANK AND NA CELLS TRUE PANDAS EMPTY CELLS AT SOME POINT
#  df.replace('NA', np.nan, inplace=True)
#
# IMPORT ALL THE NECESSARY LIBRARIES
#
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import r2_score
import sys
import os
#
# IDENTIFY THE DEFAULT PATH FOR ALL OUTPUT FILES (change the variable for path_1 to whatever your filepath is):
#
path_1 = Path(r'C:\Users\zavarin1\Documents\CODES\jaea sorption database\JAEA conversion codeR2\jaea extraction test')
#
# Pull in dataset1, tab delimited
#
df = pd.read_csv(path_1.joinpath("SDBDataDownload.txt"),sep = '\t',low_memory=False)

In [32]:
# Pull in dataset2, tab delimited
#
df1 = pd.read_csv(path_1.joinpath("SDBDataDownload-1.txt"),sep = '\t',low_memory=False)

In [33]:
# Pull in dataset3, tab delimited
#
df2 = pd.read_csv(path_1.joinpath("SDBDataDownload-2.txt"),sep = '\t',low_memory=False)

In [34]:
# Pull in dataset4, tab delimited
#
df3 = pd.read_csv(path_1.joinpath("SDBDataDownload-3.txt"),sep = '\t',low_memory=False)

In [35]:
#combine all datasets into one name
frames = [df, df1, df2, df3]

In [36]:
pd.set_option("display.max_rows", None)
#Combine all four original JAEA exported datafiles
df5 = pd.concat(frames)
#reset the index to ensure no duplicate indexing
df5.reset_index(level=None, drop=True, inplace=True, col_level=0)
#output the full list as a csv file
df5.to_csv('1.full_list.csv')
#make a file that lists statistics for solid phase group and solid phase
with open("Solid_Phase_Group_full_list.txt", "w") as f:
  print(df5['Solid Phase Group'].value_counts(), file=f, sep = "\n")
with open("Solid_Phase_full_list.txt", "w") as f:
  print(df5['Solid Phase'].value_counts(), file=f, sep = "\n")
# Select solid phase groups of interest
acceptable=['Other minerals', 'Clay minerals', 'Bentonite (smectite)']
indexNames = df5[ ~df5['Solid Phase Group'].isin(acceptable) ].index
df5.drop(indexNames, inplace=True)

In [37]:
#make a file that lists statistics for solid phase group and solid phase after screening for acceptable groups
pd.set_option("display.max_rows", None)
with open("Solid_Phase_Group_filtered.txt", "w") as f:
  print(df5['Solid Phase Group'].value_counts(), file=f, sep = "\n")
with open("Solid_Phase_filtered.txt", "w") as f:
  print(df5['Solid Phase'].value_counts(), file=f, sep = "\n")
df5.to_csv('2.filtered_list.csv')

In [38]:
#Rename the sorbate to LLNL SCIE format
#df5['redox'].replace({'I': '+1', 'II': '+2', 'III':'+3', 'IV':'+4', 'V':'+5' ,'VI':'+6', 'n.r':'n.r', 'VII':'+7', 'VIII':'+8', '-I':'-1','-II':'-2', 'V/IV': "+5,4", 'IV/V': "+4,5", 'V/VI': "+5,6"}, inplace=True)
df5['redox']=df5['redox'].replace({'I': '+1', 'II': '+2', 'III':'+3', 'IV':'+4', 'III/IV':'+3/+4', 'V':'+5' ,'VI':'+6', 'n.r.':'n.r.', 'VII':'+7', 'VIII':'+8', '-I':'-1','-II':'-2', 'V/IV': "+5/+4", 'IV/V': "+4/+5", 'V/VI': "+5/+6", 'III/IV/V/VI' : '+3/+4/+5/+6'})
#df5['redox']=df5['redox'].replace('I': '+1', inplace=True)
df5['Sorbate']=df5['Element']+'('+df5['redox'].astype(str)+')'
#df5['redox'].replace({'+1':'I', '+2':'II', '+3':'III', '+4':'IV', '+5':'V' ,'+6':'VI', 'n.r':'n.r', '+7':'VII', '+8':'VIII', '-1':'-I','-2':'-II', "+5,4":'V/IV',  "+4,5":'IV/V',  "+5,6":'V/VI'}, inplace=True)
# remove some column after processing
df5.drop('Element', inplace=True, axis=1)
df5.drop('redox', inplace=True, axis=1)
df5.to_csv('3.filtered_list_sorbate_corrected.csv')

In [39]:
#Export csv of all jaea subset database minerals and Rename Solid Phase to mineral_sources
mineral_list = list(set(df5["Solid Phase"]))
#df5['Solid Phase and note']= df5['Solid Phase']+'('+df5['note'].astype(str)+')'
#mineral_list2 = list(set(df5["Solid Phase and note"]))
df6 = pd.DataFrame(mineral_list)
df6.to_csv(path_1.joinpath('mineral_list.csv'), index = False)
df5.rename({'Solid Phase':'Mineral_sources'},axis=1,inplace=True)

In [40]:
#read file that converts jaea mineral names to consistent l-scie format: mineral_sources.csv is used to make the mienrals_conversion file
df7 = pd.read_csv(path_1.joinpath('minerals_conversion.csv'))
df7 = df7.reset_index().set_index('Mineral_source')
df5['Mineral']=df5.apply(lambda row: df7.loc[row['Mineral_sources'],'Mineral'],axis=1)
df5['Mineral_formula']=df5.apply(lambda row: df7.loc[row['Mineral_sources'],'Mineral_formula'],axis=1)
df5.rename({'Mineral_sources':'Mineral_source'},axis=1,inplace=True)
df5['Mineral_source']=df5['Mineral_source']+'('+df5['note'].astype(str)+')'
# remove any commas that are in the notes and mineral source strings
df5['Mineral_source']=df5['Mineral_source'].str.replace(',','')
df5.to_csv('4.filtered_list_mineral_corrected.csv')

In [41]:
#Rename surface area information
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.06-0.07[m2/cm3]':'0.024-0.028'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.18-0.21[m2/cm3]':'0.072-0.084'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'-':'NA'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'n.r.':'NA'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'<0.177mm':'0.01'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'<1':'9.0'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.31?}0.05':'0.31(+-)0.05'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.38+0.05':'0.38(+-)0.05'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.49?}0.05':'0.49(+-)0.05'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.5?}0.1':'0.5(+-)0.1'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.50?}0.07':'0.5(+-)0.07'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'0.69+0.13':'0.69(+-)0.13'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'1.44+0.16':'1.44(+-)0.16'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'103 (EG)':'103.0'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'11(BET)':'11.0'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'2.38+0.07':'2.38(+-)0.07'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'35(+)0.35':'35(+-)0.3'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'6.55?}0.59':'6.55(+-)0.59'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'6.68?}0.73':'6.68(+-)0.73'})
df5['Specific Surface Area(m2/g)']=df5['Specific Surface Area(m2/g)'].replace({'n.r.(<100microm)':'NA'})
df5.rename({'Specific Surface Area(m2/g)':'MineralSA'},axis=1,inplace=True)
# split surface area and SD based on the format XX(+-)YY
df5['temporary']=df5['MineralSA'].str.find(')')
df5['MineralSA_SD']=df5.apply(lambda row: row['MineralSA'][row['temporary']+1:] if row['temporary']!=-1 else 'NA',axis=1)
df5['temporary']=df5['MineralSA'].str.find('(')
df5['MineralSA']=df5.apply(lambda row: row['MineralSA'][:row['temporary']] if row['temporary']!=-1 else row['MineralSA'][:],axis=1)
df5['MineralSA_SD']=df5['MineralSA_SD'].replace({'':'NA'})
df5['MineralSA']=df5['MineralSA'].replace({'':'NA'})
# split surface area and SD based on the format XX+YY
df5['temporary']=df5['MineralSA'].str.find('+')
df5['MineralSA_SD']=df5.apply(lambda row: row['MineralSA'][row['temporary']+1:] if row['temporary']!=-1 else row['MineralSA_SD'],axis=1)
df5['MineralSA']=df5.apply(lambda row: row['MineralSA'][:row['temporary']] if row['temporary']!=-1 else row['MineralSA'][:],axis=1)
# split surface area and SD based on the format XX-YY and do some math
df5['temporary']=df5['MineralSA'].str.find('-')
df5['MineralSA_SD']=df5.apply(lambda row: ((float(row['MineralSA'][row['temporary']+1:]) - float(row['MineralSA'][:row['temporary']])) / 2) if row['temporary']!=-1 else row['MineralSA_SD'],axis=1)
df5['MineralSA']=df5.apply(lambda row: ((float(row['MineralSA'][row['temporary']+1:]) + float(row['MineralSA'][:row['temporary']])) / 2) if row['temporary']!=-1 else row['MineralSA'],axis=1)
df5['MineralSA_units']='m2/g'
df5.drop('temporary', inplace=True, axis=1)
# CONVERT TO FLOATING POINT
df5['MineralSA']=df5['MineralSA'].replace({'NA': np.nan})
df5['MineralSA_SD']=df5['MineralSA_SD'].replace({'NA': np.nan})
df5['MineralSA'] = df5['MineralSA'].astype(float)
df5['MineralSA_SD'] = df5['MineralSA_SD'].astype(float)
df5.to_csv('5.filtered_list_SA_corrected.csv')

In [42]:
#Rename CEC information
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'<0.01':'0.01'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'0.454mmol/g':'45.4'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'0.821mmol/g':'82.1'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'43mol/kg':'43.0'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'50mol/kg':'50.0'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'60mol/kg':'60.0'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'62.3?}1.2':'62.3(+-)12'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'66.5?}0.7':'66.5(+-)0.7'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'78.7?}4.8':'78.7(+-)4.8'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'79 to 85':'79-85'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'85.1?}0.1':'85.1(+-)0.1'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'85?}2.7':'85(+-)2.7'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'87(+)3.5':'87(+-)3.5'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'89?}4':'89(+-)4'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'Bentonite (smectite)':'NA'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'low CEC':'NA'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'n.r':'NA'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'n.r.':'NA'})
df5['CEC(meq/100g)']=df5['CEC(meq/100g)'].replace({'78.7(+-)48':'78.7(+-)4.8'})
df5.rename({'CEC(meq/100g)':'CEC'},axis=1,inplace=True)
df5['CEC_SD']='NA'
df5['CEC_units']='meq/100g'
# split CEC and SD based on format XX(+-)YY
df5['temporary']=df5['CEC'].str.find(')')
df5['CEC_SD']=df5.apply(lambda row: row['CEC'][row['temporary']+1:] if row['temporary']!=-1 else 'NA',axis=1)
df5['temporary']=df5['CEC'].str.find('(')
df5['CEC']=df5.apply(lambda row: row['CEC'][:row['temporary']] if row['temporary']!=-1 else row['CEC'][:],axis=1)
# split surface area and SD based on the format XX-YY and do some math
df5['temporary']=df5['CEC'].str.find('-')
df5['CEC_SD']=df5.apply(lambda row: ((float(row['CEC'][row['temporary']+1:]) - float(row['CEC'][:row['temporary']])) / 2) if row['temporary']!=-1 else row['CEC_SD'],axis=1)
df5['CEC']=df5.apply(lambda row: ((float(row['CEC'][row['temporary']+1:]) + float(row['CEC'][:row['temporary']])) / 2) if row['temporary']!=-1 else row['CEC'],axis=1)
df5.drop('temporary', inplace=True, axis=1)

# CONVERT TO FLOATING POINT
df5['CEC']=df5['CEC'].replace({'NA': np.nan})
df5['CEC_SD']=df5['CEC_SD'].replace({'NA': np.nan})
df5['CEC'] = df5['CEC'].astype(float)
df5['CEC_SD'] = df5['CEC_SD'].astype(float)


df5.to_csv('6.filtered_list_CEC_corrected.csv')

In [43]:
#Remove unwanted columns
# the note column can have useful mineral characteristics info...that info is added to the "mineral source" column
df5.drop('note', inplace=True, axis=1)
# solid phase group is the grouping of data in jaea
df5.drop('Solid Phase Group', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [44]:
#Invert the solid solution value
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['Solution/Solid(mL/g)']=df5['Solution/Solid(mL/g)'].replace({'0.3 and 13.2':'76-3333'})
df5['Solution/Solid(mL/g)']=df5['Solution/Solid(mL/g)'].replace({'0.32-13.5':'74-3125'})
df5['Solution/Solid(mL/g)']=df5['Solution/Solid(mL/g)'].replace({'20 or 80':'20-80'})
df5['Solution/Solid(mL/g)']=df5['Solution/Solid(mL/g)'].replace({'45 to 90':'45-90'})
df5['Mineral_SD']='NA'
df5['Mineral_units']='g/L'

#REMOVE ROWS THAT DO NOT HAVE EXPLICIT SSR
df5.drop(df5.loc[df5['Solution/Solid(mL/g)']=='n.r.'].index, inplace=True)

def SD_fxn (the_input):
    if 'ca.' in the_input:
        a= float(the_input[3:])
        return (1/a*1000)
    elif '-' in the_input:
        a=1/float(the_input[:the_input.index('-')])*1000
        b=1/float(the_input[the_input.index('-')+1:])*1000
        return ((a-b)/2)
    else:
        return 'NA' # np.nan #

def AVG_fxn (my_input):
    try:
        return (1/float(my_input)*1000)
    except:
        if 'ca.' in my_input:
            return 1/float(my_input[3:])*1000
        elif '-' in my_input:
            a=1/float(my_input[:my_input.index('-')])*1000
            b=1/float(my_input[my_input.index('-')+1:])*1000
            return ((b+a)/2)
        else:
            return 'NA' # np.nan #

df5['Mineral_SD']=df5.apply(lambda row: SD_fxn(row['Solution/Solid(mL/g)']),axis=1)
df5['Solution/Solid(mL/g)']=df5.apply(lambda row: AVG_fxn(row['Solution/Solid(mL/g)']),axis=1)
df5.rename({'Solution/Solid(mL/g)':'Mineral_val'},axis=1,inplace=True)


# CONVERT TO FLOATING POINT
df5['Mineral_val']=df5['Mineral_val'].replace({'NA': np.nan})
df5['Mineral_SD']=df5['Mineral_SD'].replace({'NA': np.nan})
df5['Mineral_val'] = df5['Mineral_val'].astype(float)
df5['Mineral_SD'] = df5['Mineral_SD'].astype(float)

#drop values that are 0 or blank
#df5.drop(df5.loc[df5['Mineral_val']==np.nan].index, inplace=True)


df5.dropna(subset=["Mineral_val"], inplace=True)



df5.to_csv('7.filtered_list_mineral_val_corrected.csv')

In [45]:
#drop unnecessary columns
df5.drop('Solution(mL)', inplace=True, axis=1)
df5.drop('Solid(g)', inplace=True, axis=1)

In [46]:
#Rename temperature info
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['temp(degC)']=df5['temp(degC)'].replace({'20,40,60':'40(+-)20'})
df5['temp(degC)']=df5['temp(degC)'].replace({'22?}2':'22(+-)2'})
df5['temp(degC)']=df5['temp(degC)'].replace({'25(23+-2)':'23(+-)2'})
df5['temp(degC)']=df5['temp(degC)'].replace({'25?}0.1':'25(+-)0.1'})
df5['temp(degC)']=df5['temp(degC)'].replace({'25?}1':'25(+-)1'})
df5['temp(degC)']=df5['temp(degC)'].replace({'25?}2':'25(+-)2'})
df5['temp(degC)']=df5['temp(degC)'].replace({'25+3':'25(+-)3'})
df5['temp(degC)']=df5['temp(degC)'].replace({'ambient':'25'})
df5['temp(degC)']=df5['temp(degC)'].replace({'n.r. (room)':'25'})
df5['temp(degC)']=df5['temp(degC)'].replace({'room':'25'})
df5['temp(degC)']=df5['temp(degC)'].replace({'room (22?}2??)':'25'})
#
def temp_fxn(the_reference):
    if '+' in the_reference and '-' in the_reference:
        return float(the_reference[:the_reference.index('(')])
    elif '-' in the_reference:
        a=float(the_reference[:the_reference.index('-')])
        b=float(the_reference[the_reference.index('-')+1:])
        return float((a+b)/2)
    else:
        try:
            return float(the_reference)
        except:
                return 'NA' # np.nan #
def tempsd_fxn(other_reference):
    if ')' in other_reference:
#        try:
            return float(other_reference[other_reference.index(')')+1:])
#        except:
#            return float(other_reference[-2])
    elif '-' in other_reference:
        a=float(other_reference[:other_reference.index('-')])
        b=float(other_reference[other_reference.index('-')+1:])
        return float((b-a)/2)
    else:
        return 'NA' # np.nan #

df5['Temp']=df5.apply(lambda row: temp_fxn(row['temp(degC)']),axis=1)
df5['Temp_SD']=df5.apply(lambda row: tempsd_fxn(row['temp(degC)']),axis=1)
#
#assume here that an empty value is room temperature: 25C
df5['Temp']=df5['Temp'].replace({'NA' : '25'})
#
# CONVERT TO FLOATING POINT
df5['Temp']=df5['Temp'].replace({'NA': np.nan})
df5['Temp_SD']=df5['Temp_SD'].replace({'NA': np.nan})
df5['Temp'] = df5['Temp'].astype(float)
df5['Temp_SD'] = df5['Temp_SD'].astype(float)

df5.drop('temp(degC)', inplace=True, axis=1)
df5.to_csv('8.filtered_list_temp_corrected.csv')

In [47]:
#Multistep conversion of electrolyte information
df5.replace(np.nan,'NA',inplace=True)
reference=[('Na(ppm)','Na(+1)',22.99),('K(ppm)','K(+1)',39.1),('Ca(ppm)','Ca(+2)',40.08),('Mg(ppm)','Mg(+2)',24.31),('Cl(ppm)','Cl(-1)',35.45),('NO3(ppm)','N(+5)',62.01),('ClO4(ppm)','Cl(+7)',99.45),('HCO3(ppm)','C(+4)',61.02),('SO4(ppm)','S(+6)',96.06),('F(ppm)','F(-1)',19.0),('SiO2(ppm)','Si(+4)',60.09),('Fe(ppm)','Fe(+3)',55.84),('DOC(ppm)','DOC',12.01)]

def a_fxn(a,b):
    if a=='NA':
        return 'NA'
    else:
        return b

for f in range(1,len(reference)+1):
    df5["Electrolyte"+str(f)+"_val"]=df5.apply(lambda row: row[reference[f-1][0]]/reference[f-1][2]/1000 if row[reference[f-1][0]] != 'NA' else 'NA',axis=1)
    df5["Electrolyte"+str(f)]=reference[f-1][1]
    df5["Electrolyte"+str(f)+"_SD"]='NA'
    df5["Electrolyte"+str(f)+"_units"]='mol/L'
    df5["Electrolyte"+str(f)+"_val"]=df5["Electrolyte"+str(f)+"_val"].replace({'' : 'NA'})
    df5["Electrolyte"+str(f)]=df5.apply(lambda row: a_fxn(row["Electrolyte"+str(f)+"_val"], row["Electrolyte"+str(f)]), axis=1)
    df5["Electrolyte"+str(f)+"_units"]=df5.apply(lambda row: a_fxn(row["Electrolyte"+str(f)+"_val"], row["Electrolyte"+str(f)+"_units"]), axis=1)
    df5.drop(reference[f-1][0], inplace=True, axis=1)
df5.to_csv('9.filtered_list_electrolyte_corrected.csv')

In [48]:
#Rename pH info and export csv with all pH values
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['pH end']=df5['pH end'].replace({'n.r.':'NA'})
df5['pH init']=df5['pH init'].replace({'n.r.':'NA'})
df5['pH end']=df5['pH end'].replace({'':'NA'})
df5['pH init']=df5['pH init'].replace({'':'NA'})
# integrate init and end data into one pH column
df5['pH']=df5.apply(lambda row: row['pH end'] if row['pH end'] != 'NA' else row['pH init'],axis=1)
df5.drop('pH end', inplace=True, axis=1)
df5.drop('pH init', inplace=True, axis=1)
df5['pH_SD']='NA'
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['pH']=df5['pH'].replace({'6.9-8.6 as pHc':'6.9-8.6'})
df5['pH']=df5['pH'].replace({'6.2 as pHc':'6.2'})
df5['pH']=df5['pH'].replace({'7.2 as pHc':'7.2'})
df5['pH']=df5['pH'].replace({'7.2?}0.2':'7.2(+-)0.2'})
df5['pH']=df5['pH'].replace({'7.6 as pHc':'7.6'})
df5['pH']=df5['pH'].replace({'7.7 as pHc':'7.7'})
df5['pH']=df5['pH'].replace({'7.8 as pHc':'7.8'})
df5['pH']=df5['pH'].replace({'8.2 as pHc':'8.2'})
df5['pH']=df5['pH'].replace({'9.2 as pHc':'9.2'})
df5['pH']=df5['pH'].replace({'9.7 as pHc':'9.7'})
df5['pH']=df5['pH'].replace({'ca. 2-6':'2-6'})
df5['pH']=df5['pH'].replace({'ca. 7-11':'7-11'})
def pH_fxn(my_input):
    if '(' in my_input:
        return float(my_input[:my_input.index('(')])
    elif '-' in my_input:
        a=float(my_input[:my_input.index('-')])
        b=float(my_input[my_input.index('-')+1:])
        return float((a+b)/2)
    else:
        return my_input

def pH_SD_fxn(same_input,original):
    if ')' in same_input:
        return float(same_input[same_input.index(')')+1:])
    elif '-' in same_input:
        a= float(same_input[:same_input.index('-')])
        b=float(same_input[same_input.index('-')+1:])
        return float((b-a)/2)
    else:
        return original

df5['pH_SD']=df5.apply(lambda row: pH_SD_fxn(row['pH'],row['pH_SD']),axis=1)
df5['pH']=df5.apply(lambda row: pH_fxn(row['pH']),axis=1)

# CONVERT TO FLOATING POINT
df5['pH']=df5['pH'].replace({'NA': np.nan})
df5['pH_SD']=df5['pH_SD'].replace({'NA': np.nan})
df5['pH'] = df5['pH'].astype(float)
df5['pH_SD'] = df5['pH_SD'].astype(float)

df5.to_csv('10.filtered_list_pH_corrected.csv')

In [49]:
#Rename Sorbate concentration info
#CORRECT SOME JAEA DATABASE ERRORS/typos
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'(1.2-1.3)E-07':'1.2E-7 - 1.3E-07'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'1.00E-12 or 1.40E-12':'1.00E-12 - 1.40E-12'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'1.30E-05 (150Sm)':'1.30E-05'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'2.60E-05 (150Sm)':'2.60E-05'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'4.00e-10 and 1.00e-06':'4.00e-10 - 1.00e-06'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'5.30E-06 (150Sm)':'5.30E-06'})
#df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'carrier-free':'0.0'})
#df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'NA':'0.0'})
#df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'n.r.':'0.0'})
df5['C init(mol/L)']=df5['C init(mol/L)'].replace({'1.00E-04 - 1.00E-09':'1.00E-09 - 1.00E-04'})
#REMOVE ROWS THAT DO NOT HAVE EXPLICIT SORBATE CONCENTRATION LISTED
df5.drop(df5.loc[df5['C init(mol/L)']=='carrier-free'].index, inplace=True)
df5.drop(df5.loc[df5['C init(mol/L)']=='n.r.'].index, inplace=True)

def sorbate_SD_fxn(my_input):
    if '<' in my_input:
        return float(my_input[1:])
    elif ' - ' in my_input:
        a=float(my_input[:my_input.index(' - ')])
        b=float(my_input[my_input.index(' - ')+3:])
        return (b-a)/2
    else:
        a=float(my_input)
        return a*0.05

def sorbate_fxn(same_input):
    if '<' in same_input:
        return float(same_input[1:])
    elif ' - ' in same_input:
        a=float(same_input[:same_input.index(' - ')])
        b=float(same_input[same_input.index(' - ')+3:])
        return (a+b)/2
    else:
        return same_input

df5.rename({'C init(mol/L)':'Sorbate_val'},axis=1,inplace=True)
df5['Sorbate_SD']='NA'
df5['Sorbate_SD']=df5.apply(lambda row: sorbate_SD_fxn(row['Sorbate_val']),axis=1)
df5['Sorbate_val']=df5.apply(lambda row: sorbate_fxn(row['Sorbate_val']),axis=1)
df5['Sorbate_units']='mol/L'
#now convert columns into floating point rather than text
df5['Sorbate_val'] = df5['Sorbate_val'].astype(float)
df5['Sorbate_SD'] = df5['Sorbate_SD'].astype(float)
# remove Sorbate_val rows that are equal to 0 
df5.drop(df5.loc[df5['Sorbate_val']== 0].index, inplace=True)
df5.to_csv('11.filtered_list_sorbate_corrected.csv')

In [50]:
#Remove columns
df5.drop('Contact time(day)', inplace=True, axis=1)
df5.drop('Separation', inplace=True, axis=1)
df5.drop('Eh init(mV)', inplace=True, axis=1)
df5.drop('Eh end(mV)', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [51]:
#Export csv file of all atm/redox conditions in the jaea subset file; used to convert atm conditions in jaea to l-scie
atm_conditions = list(set(df5["atm/redox condition"]))
df10 = pd.DataFrame(atm_conditions)
df10.to_csv(path_1.joinpath('atm_conditions.csv'), index = False)

In [52]:
#Multistep conversion of gas composition info from jaea to l-scie format
df6 = pd.read_csv(path_1.joinpath("atm_conditions_conversion.txt"),sep = '\t')
df6 = df6.reset_index().set_index('jaea: atm/redox condition')
other_reference=['Gas1','Gas1_val','Gas1_SD','Gas1_units','Gas2','Gas2_val','Gas2_SD','Gas2_units','Gas3','Gas3_val','Gas3_SD','Gas3_units']
for g in other_reference:
    df5[g]=df5.apply(lambda row: df6.loc[row['atm/redox condition'],g] if row['atm/redox condition'] != 'NA' else 'NA',axis=1)
df5.replace(np.nan, 'NA',inplace=True)
df5.to_csv('12.filtered_list_gas_corrected.csv')

In [53]:
#Remove columns
df5.drop('Note', inplace=True, axis=1)
df5.drop('replicates, n', inplace=True, axis=1)
df5.drop('additional/information', inplace=True, axis=1)
df5.drop('Kd Graph Data', inplace=True, axis=1)
df5.drop('Temp Graph Data', inplace=True, axis=1)
df5.drop('pH Graph Data', inplace=True, axis=1)
df5.drop('Time Graph Data', inplace=True, axis=1)
df5.drop('Eh Graph Data', inplace=True, axis=1)
df5.drop('Cinit Graph Data', inplace=True, axis=1)
df5.drop('Solution/Solid Graph Data', inplace=True, axis=1)
df5.drop('Data type', inplace=True, axis=1)
#df5.to_csv('df5.csv')

In [54]:
#SPECIFIC CORRECTIONS TO JAEA DATABASE BASED ON LLNL QA OF THE DATABASE INFORMATION
#
# CORRECT THE GORGEON KD ERROR IN THE DATABASE THAT DOES NOT MAKE PHYSICAL SENSE...REPLACE WITH ESTIMATED KD ERROR
# ALSO FIX GAS COMPOSITIONS AS THE PAPER CLEARLY STATES NO EQUILIBRIUM WITH CO2 - USE 1e-5 MOL/L C(+4) IS RECOMMENDED
df5['error']= df5.apply(lambda row: 0.58*float(row['Kd(m3/kg)']) if row['Reference']=='Gorgeon(1994)'  else row['error'],axis=1)
df5['Electrolyte2']= df5.apply(lambda row: 'C(+4)' if row['Reference']=='Gorgeon(1994)'  else row['Electrolyte2'],axis=1)
df5['Electrolyte2_val']= df5.apply(lambda row: 1E-5 if row['Reference']=='Gorgeon(1994)'  else row['Electrolyte2_val'],axis=1)
df5['Electrolyte2_units']= df5.apply(lambda row: 'mol/L' if row['Reference']=='Gorgeon(1994)'  else row['Electrolyte2_units'],axis=1)
df5['Electrolyte2_SD']= df5.apply(lambda row: 'NA' if row['Reference']=='Gorgeon(1994)'  else row['Electrolyte2_SD'],axis=1)
df5['Gas3']= df5.apply(lambda row: 'NA' if row['Reference']=='Gorgeon(1994)'  else row['Gas3'],axis=1)
df5['Gas3_val']= df5.apply(lambda row: 'NA' if row['Reference']=='Gorgeon(1994)'  else row['Gas3_val'],axis=1)
df5['Gas3_SD']= df5.apply(lambda row: 'NA' if row['Reference']=='Gorgeon(1994)'  else row['Gas3_SD'],axis=1)
df5['Gas3_units']= df5.apply(lambda row: 'NA' if row['Reference']=='Gorgeon(1994)'  else row['Gas3_units'],axis=1)
df5['Sorbate']= df5.apply(lambda row: 'Np(+5)' if row['Reference']=='Gorgeon(1994)' and row['Sorbate']=='Np(n.r.)' else row['Sorbate'],axis=1)
#
ref_extract_r1 = df5[df5["Reference"]=='Gorgeon(1994)']
ref_extract_r1.to_csv('ref_extract_r1.csv')

In [55]:
#Multref_extract = df5[df5["Reference"]=='Gorgeon(1994)']istep conversion of jaea: kd column
#various corrections and typos in jaea database
df5['Kd(m3/kg)']=df5['Kd(m3/kg)'].replace({'n.r.':'NA'})
df5['error']=df5['error'].replace({'n.r.':'NA'})


def last_fxn(my_input):
    a=float(my_input[:my_input.index(' - ')])
    b=float(my_input[my_input.index(' - ')+3:])
    return (a+b)/2
def KdSD_fxn(my_input):
    a=float(my_input[:my_input.index(' - ')])
    b=float(my_input[my_input.index(' - ')+3:])
    return (b-a)/2
# identify if Kd has a > or < in front of the value
df5['check']=df5.apply(lambda row: True if ('>' in row['Kd(m3/kg)'] or '<' in row['Kd(m3/kg)']) else False,axis=1)
# remove < or > from the Kd
df5['Kd(m3/kg)']=df5.apply(lambda row: float(row['Kd(m3/kg)'][1:]) if row['check'] is True else row['Kd(m3/kg)'],axis=1)
# set the error for Kds with > or < as 100% SD
df5['error']=df5.apply(lambda row: float(row['Kd(m3/kg)']) if row['check'] is True else row['error'],axis=1)
# set the error for Kds that are listed as a range to 1/2 of the range
df5['error']=df5.apply(lambda row: KdSD_fxn(row['Kd(m3/kg)']) if ' - ' in str(row['Kd(m3/kg)']) else row['error'],axis=1)
# set the Kd that is listed as a range to the average value
df5['Kd(m3/kg)']=df5.apply(lambda row: last_fxn(row['Kd(m3/kg)']) if ' - ' in str(row['Kd(m3/kg)']) else row['Kd(m3/kg)'],axis=1)
# for Kds listed as "-", we simply set them to 0
df5['Kd(m3/kg)']=df5.apply(lambda row: float(row['Kd(m3/kg)'] == '0.0') if row['Kd(m3/kg)'] == '-' else row['Kd(m3/kg)'],axis=1)
# if the error is listed as 0.2log or 0.5log, we calculated the error 
df5['error']= df5.apply(lambda row: 10**(math.log10(float(row['Kd(m3/kg)']))+0.2)-float(row['Kd(m3/kg)']) if (row['error']=='0.2log' or row['error']=='?}0.2log') else row['error'],axis=1)
df5['error']= df5.apply(lambda row: 10**(math.log10(float(row['Kd(m3/kg)']))+0.5)-float(row['Kd(m3/kg)']) if row['error']=='0.5log' else row['error'],axis=1)
#  error with NA or "-" or 0 or '' needs to have default error...we set this to 0.2 log units ~ 0.58Kd
df5['error']= df5.apply(lambda row: 0.58*float(row['Kd(m3/kg)']) if row['error']=='-'  else row['error'],axis=1)
df5['error']= df5.apply(lambda row: 0.58*float(row['Kd(m3/kg)']) if row['error']==''  else row['error'],axis=1)
df5['error']= df5.apply(lambda row: 0.58*float(row['Kd(m3/kg)']) if row['error']== 'NA'  else row['error'],axis=1)
# error converted from jaea to l-scie units for specific terms
df5['error']= df5.apply(lambda row: 0.05*float(row['Kd(m3/kg)']) if (row['error']=='?`5%' or row['error']=='5%') else row['error'],axis=1)

#fix the zero or negative Kd values to nominal low value and set error the same way
df5['Kd(m3/kg)'] = df5['Kd(m3/kg)'].astype(float)
df5['error'] = df5['error'].astype(float)
df5.loc[df5['Kd(m3/kg)'] <= 0, 'error'] = 0.58E-6
df5.loc[df5['Kd(m3/kg)'] <= 0, 'Kd(m3/kg)'] = 1.0E-6
df5['error']= df5.apply(lambda row: 0.58*float(row['Kd(m3/kg)']) if row['error']== 0  else row['error'],axis=1)

#calculate aqueous concentration from Kd, total sorbate, and solid solution ratio
df5['Aq_val'] = df5.apply(lambda row: float(row['Sorbate_val']) / (float(row['Kd(m3/kg)']) * float(row['Mineral_val']) + 1) \
                if ((row['Sorbate_val']) != 'NA' and float(row['Sorbate_val']) != 0 and \
                    (row['Kd(m3/kg)']) != 'NA' and float(row['Kd(m3/kg)']) != 0 and \
                    (row['Mineral_val']) != 'NA' and float(row['Mineral_val']) != 0 ) else 'NA',axis=1)

#calculate error in aqueous value based on uncertainty in Kd
df5['Aq_SD'] = df5.apply(lambda row: (((float(row['error'])*float(row['Mineral_val'])/(float(row['Kd(m3/kg)'])*float(row['Mineral_val'])+1))**2 \
                +(float(row['Sorbate_SD'])/float(row['Sorbate_val']))**2)**0.5*(row['Aq_val'])) \
                if ((row['Sorbate_val']) != 'NA' and float(row['Sorbate_val']) != 0 and \
                    (row['Kd(m3/kg)']) != 'NA' and float(row['Kd(m3/kg)']) != 0 and \
                    (row['Mineral_val']) != 'NA' and float(row['Mineral_val']) != 0 and \
                    (row['Sorbate_SD']) != 'NA' and float(row['Sorbate_SD']) != 0 and \
                    (row['Aq_val']) != 'NA' and float(row['Aq_val']) != 0 and \
                    (row['error']) != 'NA' and float(row['error']) != 0 ) else 'NA',axis=1)
df5['Aq_units']='mol/L'

#Calculate sorbed concentration (mol/L) based on Kd, total sorbate, and solid solution ratio
df5['Sorbed_val'] = df5.apply(lambda row: float(row['Sorbate_val'])*float(row['Mineral_val']) / ((1/float(row['Kd(m3/kg)'])) + float(row['Mineral_val'])) \
                if ((row['Sorbate_val']) != 'NA' and float(row['Sorbate_val']) != 0 and \
                    (row['Kd(m3/kg)']) != 'NA' and float(row['Kd(m3/kg)']) != 0 and \
                    (row['Mineral_val']) != 'NA' and float(row['Mineral_val']) != 0 ) else 'NA',axis=1)

#calculate error in sorbed value based on uncertainty in Kd
df5['Sorbed_SD'] = df5.apply(lambda row: (((float(row['error'])/(float(row['Kd(m3/kg)'])+float(row['Mineral_val'])*(float(row['Kd(m3/kg)']))**2))**2 \
                +(float(row['Sorbate_SD'])/float(row['Sorbate_val']))**2)**0.5*(row['Sorbed_val'])) \
                if ((row['Sorbate_val']) != 'NA' and float(row['Sorbate_val']) != 0 and \
                    (row['Kd(m3/kg)']) != 'NA' and float(row['Kd(m3/kg)']) != 0 and \
                    (row['Mineral_val']) != 'NA' and float(row['Mineral_val']) != 0 and \
                    (row['Sorbate_SD']) != 'NA' and float(row['Sorbate_SD']) != 0 and \
                    (row['Sorbed_val']) != 'NA' and float(row['Sorbed_val']) != 0 and \
                    (row['error']) != 'NA' and float(row['error']) != 0 ) else 'NA',axis=1)
df5['Sorbed_units']='mol/L'

df5.drop('Kd(m3/kg)', inplace=True, axis=1)
df5.drop('error', inplace=True, axis=1)
df5.drop('check', inplace=True, axis=1)

#now convert columns into floating point rather than text
df5['Aq_val'] = df5['Aq_val'].astype(float)
df5['Aq_SD'] = df5['Aq_SD'].astype(float)
df5['Sorbed_val'] = df5['Sorbed_val'].astype(float)
df5['Sorbed_SD'] = df5['Sorbed_SD'].astype(float)

df5.to_csv('13.filtered_list_Kd_corrected.csv')

In [56]:
#Conversion of reference columns
df5['Year']=df5['Year'].astype(int)
df5['Ref_Combo']=df5['Author'].astype(str) + df5['Year'].astype(str) + df5['Title'].astype(str) + df5['Journal'].astype(str)
#Export csv of all jaea references
reference_sources = list(set(df5["Ref_Combo"]))
df_Ref_Combo = pd.DataFrame(reference_sources)
df_Ref_Combo.to_csv(path_1.joinpath('reference_sources.csv'), index = False)
# build lookup table for defining reference name and DOI, insert into main dataframe
df8 = pd.read_csv(path_1.joinpath("doi_id_reference.txt"), sep = '\t', low_memory=False)
df8 = df8.reset_index().set_index('Combo')
df5['Reference']=df5.apply(lambda row: df8.loc[row['Ref_Combo'],'LLNL Reference'],axis=1)
df5['doi_link']=df5.apply(lambda row: df8.loc[row['Ref_Combo'],'DOI Link'],axis=1)
df5.drop('Save No', inplace=True, axis=1)
df5.drop('Water Type', inplace=True, axis=1)
df5.drop('Ionic strength(mol/L)', inplace=True, axis=1)
df5.drop('note.1', inplace=True, axis=1)
df5.drop('Ionic strength(mol/L).1', inplace=True, axis=1)
df5.drop('atm/redox condition', inplace=True, axis=1)
df5.drop('Author', inplace=True, axis=1)
df5.drop('Year', inplace=True, axis=1)
df5.drop('Title', inplace=True, axis=1)
df5.drop('Journal', inplace=True, axis=1)
df5.drop('Publisher', inplace=True, axis=1)
df5.drop('Vol', inplace=True, axis=1)
df5.drop('No', inplace=True, axis=1)
df5.drop('Page', inplace=True, axis=1)
df5.drop('Unnamed: 59', inplace=True, axis=1)
df5.to_csv('14.filtered_list_Ref_corrected.csv')

In [57]:
#Creating unique IDs
df5['Set']=df5['Reference']+'_1'
df5.sort_values('Reference')
df5['Helper']=df5['Reference'].shift()
df5.fillna(0)
df5['Check_number']=df5.apply(lambda row: 1 if row['Reference']==row['Helper'] else 0,axis=1)
ls = []  
cumulative = 0
count = 0
for _, row in df5.iterrows():
    if row.Check_number == 1:
        cumulative += row.Check_number
    else:
        cumulative = 0
        count+=1
    ls.append(cumulative)
df5['Check_number']=ls
df5['Check_number']+=1
df5['SetID']=df5.apply(lambda row: row['Reference']+'_'+str(row['Check_number']),axis=1)
df5.drop('Helper', inplace=True, axis=1)
df5.drop('Check_number', inplace=True, axis=1)
df5['doi_link']=df5['doi_link'].replace({np.nan: 'NA'})
df5.to_csv('15.filtered_list_Ref_IDcorrected.csv')

In [58]:
# Add some more columns and make consistent with Nick's code
df5['SurfCharge_val']='NA'
df5['SurfCharge_units'] ='NA'
df5['SurfCharge_SD']='NA'
df5['Mineralsites_SD']='NA'
df5['Mineralsites_units']='sites/nm2'
df5['Mineralsites']='NA'
df5 = df5[['Reference' , 'doi_link', 'Set' , 'SetID' , 'Mineral' , 'Mineral_formula' , 'Mineral_source' , 'Temp' , 'Temp_SD' , 'Electrolyte1' , 'Electrolyte1_val' , 'Electrolyte1_SD' , 'Electrolyte1_units' , 'Electrolyte2' , 'Electrolyte2_val' , 'Electrolyte2_SD' , 'Electrolyte2_units' , 'Electrolyte3' , 'Electrolyte3_val' , 'Electrolyte3_SD' , 'Electrolyte3_units' , 'Electrolyte4' , 'Electrolyte4_val' , 'Electrolyte4_SD' , 'Electrolyte4_units' , 'Electrolyte5' , 'Electrolyte5_val' , 'Electrolyte5_SD' , 'Electrolyte5_units' , 'Electrolyte6' , 'Electrolyte6_val' , 'Electrolyte6_SD' , 'Electrolyte6_units' , 'Electrolyte7' , 'Electrolyte7_val' , 'Electrolyte7_SD' , 'Electrolyte7_units' , 'Electrolyte8' , 'Electrolyte8_val' , 'Electrolyte8_SD' , 'Electrolyte8_units' , 'Electrolyte9' , 'Electrolyte9_val' , 'Electrolyte9_SD' , 'Electrolyte9_units' , 'Electrolyte10' , 'Electrolyte10_val' , 'Electrolyte10_SD' , 'Electrolyte10_units' , 'Electrolyte11' , 'Electrolyte11_val' , 'Electrolyte11_SD' , 'Electrolyte11_units' , 'Electrolyte12' , 'Electrolyte12_val' , 'Electrolyte12_SD' , 'Electrolyte12_units' , 'Electrolyte13' , 'Electrolyte13_val' , 'Electrolyte13_SD' , 'Electrolyte13_units' , 'pH' , 'pH_SD' , 'Sorbate' , 'Sorbate_val' , 'Sorbate_SD' , 'Sorbate_units' , 'Aq_val' , 'Aq_SD' , 'Aq_units' , 'Sorbed_val' , 'Sorbed_SD' , 'Sorbed_units' , 'SurfCharge_val' , 'SurfCharge_SD' , 'SurfCharge_units' , 'Mineral_val' , 'Mineral_SD' , 'Mineral_units' , 'MineralSA' , 'MineralSA_SD' , 'MineralSA_units' , 'Mineralsites' , 'Mineralsites_SD' , 'Mineralsites_units' , 'CEC' , 'CEC_SD' , 'CEC_units' , 'Gas1' , 'Gas1_val' , 'Gas1_SD' , 'Gas1_units' , 'Gas2' , 'Gas2_val' , 'Gas2_SD' , 'Gas2_units' , 'Gas3' , 'Gas3_val' , 'Gas3_SD' , 'Gas3_units']]
#df5=df5.replace({'NA': np.nan})

In [59]:
#add new column to merge with l-scie AND SET ALL AQ, S, TOT TO BE EQUAL TO OR GREATER THAN THE SD
df5['Source']='JAEA'
df5.replace(np.nan, 'NA',inplace=True)



def SD_LIMIT(VAL,SD):
    if VAL <= 0:
        return SD
    else:
        return VAL

df5['Sorbate_val']=df5.apply(lambda row: SD_LIMIT(row['Sorbate_val'],row['Sorbate_SD']),axis=1)
df5['Sorbed_val']=df5.apply(lambda row: SD_LIMIT(row['Sorbed_val'],row['Sorbed_SD']),axis=1)
df5['Aq_val']=df5.apply(lambda row: SD_LIMIT(row['Aq_val'],row['Aq_SD']),axis=1)

df5.to_csv(path_1.joinpath('jaea.csv'), index = False)